See `./project/dataset.py` for my code that generates the main dataset.

# Latent Dirichlet Allocation

In [6]:
import pandas
import numpy
import pickle
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

/home/dan/.guix-profile/lib/python3.7/site-packages/funcy/colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
/home/dan/.guix-profile/lib/python3.7/site-packages/funcy/colls.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable, Iterator, defaultdict
/home/dan/.guix-profile/lib/python3.7/site-packages/funcy/seqs.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import defaultdict, deque, Sequence
/home/dan/.guix-profile/lib/python3.7/site-packages/funcy/flow.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of f

In [9]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
numpy.random.seed(0xD00D5) 

In [11]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])

In [12]:
minimal_dataset.head()

,repo,language,documents
0,28457823,javascript,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,"b""import { Observable } from 'rx';\n\nmodule.e..."


In [13]:
documents = minimal_dataset['documents']
documents.head()

0    b"module.exports = {\n  plugins: [\n    requir...
1    b"// The path where to mount the REST API app\...
2    b"import { Observable } from 'rx';\nimport deb...
3    b"import { Observable } from 'rx';\n// import ...
4    b"import { Observable } from 'rx';\n\nmodule.e...
Name: documents, dtype: object

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").

In [14]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [15]:
len(tf_feature_names)

284872

We have four programming languages, try to use LDA to determine these four programming languages.

In [29]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)

In [30]:
model = lda.fit(tf)

/home/dan/.guix-profile/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/home/dan/.guix-profile/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=4,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
model = _

In [32]:
model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=4,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [33]:
model.components_

array([[2.38928456e+02, 1.48743859e+02, 6.63647964e+01, ...,
        2.50756970e-01, 2.50756970e-01, 2.50756970e-01],
       [2.99244369e+02, 2.57844111e-01, 3.16097064e-01, ...,
        2.50141731e-01, 2.50141731e-01, 2.50141731e-01],
       [3.75245861e+01, 2.66373116e-01, 2.91116932e-01, ...,
        2.50002502e-01, 2.50002502e-01, 2.50002502e-01],
       [2.66187753e-01, 6.41071354e+00, 2.59056078e-01, ...,
        1.38201933e+00, 1.38201933e+00, 1.38201933e+00]])

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

In [34]:
with open('../data/minimal_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

**TODO** Next, I reckon stick this in pyldavis and we can get a nice visualisation like in the DST workshop. Maybe after that we can try copy to compare the topics against programming languages and see if there was any correlation?

In [3]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [7]:
model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=4,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [15]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

/home/dan/.guix-profile/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.170498  0.068422       1        1  53.661148
3      0.334580 -0.137167       2        1  17.800762
1     -0.215109 -0.203095       3        1  16.993209
2      0.051028  0.271840       4        1  11.544881, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
224987  Default  76787.000000           self  76787.000000  30.0000  30.0000
209790  Default  28422.000000          react  28422.000000  29.0000  29.0000
118970  Default  35518.000000           from  35518.000000  28.0000  28.0000
257199  Default  93096.000000           this  93096.000000  27.0000  27.0000
256596  Default  71532.000000            the  71532.000000  26.0000  26.0000
138187  Default  75532.000000             if  75532.000000  25.0000  25.0000
93741   Default  26333.000000        default  26333.000000  24.0000  24.0000
93727   Default  19823.000000            def  19823.000000  23.0000  23.0000
119762  Default  78705.000000       function  78705.000000  22.0000  22.0000
139197  Default  26520.000000         import  26520.000000  21.0000  21.0000
180274  Default  18618.000000        nimport  18618.000000  20.0000  20.0000
178605  Default  13422.000000        nexport  13422.000000  19.0000  19.0000
6203    Default  19257.000000             12  19257.000000  18.0000  18.0000
179191  Default  15713.000000          nfrom  15713.000000  17.0000  17.0000
88624   Default  12159.000000  createsvgicon  12159.000000  16.0000  16.0000
195902  Default  18062.000000           path  18062.000000  15.0000  15.0000
182715  Default  21713.000000           none  21713.000000  14.0000  14.0000
270897  Default  16822.000000           user  16822.000000  13.0000  13.0000
10108   Default  14036.000000             16  14036.000000  12.0000  12.0000
216978  Default  74742.000000         return  74742.000000  11.0000  11.0000
139409  Default  34335.000000             in  34335.000000  10.0000  10.0000
273107  Default  50512.000000            var  50512.000000   9.0000   9.0000
58561   Default  16313.000000             as  16313.000000   8.0000   8.0000
118218  Default   8955.000000       fragment   8955.000000   7.0000   7.0000
4646    Default  16616.000000             10  16616.000000   6.0000   6.0000
281843  Default   7860.000000            xd0   7860.000000   5.0000   5.0000
16221   Default  10334.000000             24  10334.000000   4.0000   4.0000
80378   Default   8276.000000      classname   8276.000000   3.0000   3.0000
281885  Default   7548.000000            xe0   7548.000000   2.0000   2.0000
173909  Default  23687.000000           name  23687.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
281761   Topic4   1464.391410            xbd   1487.025518   2.1436  -6.2404
281631   Topic4   1663.439660            xb4   1699.407348   2.1375  -6.1129
281809   Topic4   2187.518169            xc3   2248.312349   2.1315  -5.8390
209790   Topic4  26136.460497          react  28422.863801   2.0751  -3.3585
281786   Topic4   1213.602554            xbf   1241.477370   2.1362  -6.4282
80378    Topic4   7546.761037      classname   8276.007692   2.0667  -4.6007
281561   Topic4   1517.149330            xb0   1563.031406   2.1291  -6.2050
281345   Topic4   1061.601815            xa5   1081.515369   2.1403  -6.5620
205348   Topic4   4516.349076      proptypes   4899.945261   2.0774  -5.1141
280778   Topic4   2783.143067            x80   2961.846853   2.0967  -5.5982
281673   Topic4   2447.870929            xb8   2614.659009   2.0930  -5.7266
125631   Topic4   1890.410526        gettext   2012.120308   2.0965  -5.9850
118970   Topic4  25303.791250           from  35518.688459   1.8198  -3.3908
159703   Topic4   3531.937969             m0   4032.249447   2.0265  -5.3600
180274   Topic

# Building documents as mixtures of programming languages

In [16]:
documents[0]


'b"module.exports = {\\n  plugins: [\\n    require.resolve(\'babel-plugin-transform-function-bind\'),\\n    require.resolve(\'@babel/plugin-proposal-class-properties\'),\\n    require.resolve(\'@babel/plugin-proposal-object-rest-spread\'),\\n  ],\\n  presets: [\\n    [\\n      require.resolve(\'@babel/preset-env\'),\\n      {\\n        targets: {\\n          node: \'10\',\\n        },\\n      },\\n    ],\\n  ],\\n};\\n"'